In [3]:
from twython import Twython

In [4]:
from time import sleep
import json
import pandas as pd
import io
from tqdm import tqdm

In [17]:
if not os.path.exists(os.path.dirname(tweets_clean_file)):
    try:
        os.makedirs(os.path.dirname(tweets_clean_file))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise
import os
if not os.path.exists(os.path.dirname(tweets_raw_file)):
    try:
        os.makedirs(os.path.dirname(tweets_raw_file))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [2]:
# Define the currency
#CURRENCY = "zilliqa"
#CURRENCY_SYMBOL = "ZIL"
#CURRENCY = "nexo"
#CURRENCY_SYMBOL = "NEXO"
CURRENCY = "bitcoin"
CURRENCY_SYMBOL = "BTC"

## personal config
TWEETS_FOLDER    = f"data/crypto/{CURRENCY_SYMBOL}" # Relative path to historical data
SEP_CHAR         = '~' # character seperating dates from and to in filename
ENVS             = ['CRYPTO', 'LINE_COUNT', 'MOST_RECENT_FILE', 'MOST_RECENT_ID'] # Stored in var.csv
MAX_ROW_PER_FILE = 20000 # Each file storing data has a maximum amount of rows

tweets_raw_file = f'data/twitter/{CURRENCY_SYMBOL}/{CURRENCY}_tweets_raw.csv'
tweets_clean_file = f'data/twitter/{CURRENCY_SYMBOL}/{CURRENCY}_tweets_clean.csv'
query = f'#{CURRENCY} OR #{CURRENCY_SYMBOL}' ####TODO PUT BACK  OR {CURRENCY} OR ${CURRENCY} OR ${CURRENCY_SYMBOL}

In [13]:
#The API authentication
APP_KEY = ''
APP_SECRET = ''
twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 450, 'remaining': 450, 'reset': 1588431649}}

In [15]:
#Searching for queries and storing it into CSV
NUMBER_OF_QUERIES = 450
data = {"statuses": []}
next_id = None
with open(tweets_raw_file,mode = "a+", encoding='utf-8') as f:
    if not next_id:
        f.write("ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt\n")
    while(True):
        twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)
        last_size = 0
        for i in tqdm(range(NUMBER_OF_QUERIES)):
            if not next_id:
                data = twitter.search(q=query, lang='en', result_type='recent', count="100") # Use since_id for tweets after id
            else:
                data["statuses"].extend(twitter.search(q=query, lang='en', result_type='mixed', count="100", max_id=next_id)["statuses"])
            if len(data["statuses"]) > 1:
                next_id = data["statuses"][len(data["statuses"]) - 1]['id']
            if last_size + 1 == len(data["statuses"]):
                break
            else:
                last_size = len(data["statuses"])

        print('Retrieved {0}, waiting for 15 minutes until next queries'.format(len(data["statuses"])))
        d = pd.DataFrame([[s["id"], s["text"].replace('\n','').replace('\r',''), s["user"]["name"], s["user"]["followers_count"], s["retweet_count"], s["favorite_count"], s["created_at"]] for s in data["statuses"]], columns=('ID', 'Text', 'UserName', "UserFollowerCount", 'RetweetCount', 'Likes', "CreatedAt"))
        d.to_csv(f, mode='a', encoding='utf-8',index=False,header=False)
        if last_size + 1 == len(data["statuses"]):
            print('No more new tweets, stopping...')
            break
        data["statuses"] = []
        
        sleep(910)

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [05:25<00:00,  1.38it/s]


Retrieved 43250, waiting for 15 minutes until next queries


KeyboardInterrupt: 

In [5]:
d = pd.read_csv(tweets_raw_file)

In [6]:
d.head()

,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
0,ID,Text,UserName,UserFollowerCount,RetweetCount,Likes,CreatedAt
1,1256595810265051136,Become A #Bitcoin Developer: Basic 101 https:/...,Naked Wicks,22,0,0,Sat May 02 14:46:02 +0000 2020
2,1256595809077874688,⬆️⬆️ $BTC BUYING PRESSURE ALERT 📈 Price tradin...,CryptoSquawk,284,0,0,Sat May 02 14:46:01 +0000 2020
3,1256595805978284032,RT @lil_cryp: I'm embarrassed to say I shorted...,kjvbbnnn,0,5,0,Sat May 02 14:46:01 +0000 2020
4,1256595802736046081,RT @HuobiStats: #HPT Price USDT: 0.00588 (-0.4...,kim,67,27,0,Sat May 02 14:46:00 +0000 2020


In [ ]:
# regular expressions and data cleaning
import re 
from tqdm import tnrange, tqdm_notebook, tqdm

d = pd.read_csv(tweets_raw_file)
for i,s in enumerate(tqdm(d['Text'])):
    text = d.loc[i, 'Text']
    text = text.replace("#", "")
    text = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)
    text = re.sub('@\\w+ *', '', text, flags=re.MULTILINE)
    d.loc[i, 'Text'] = text

In [ ]:
#Storing the clean file
d.to_csv(tweets_clean_file,header=True)